In [ ]:
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import liana as li
%matplotlib inline
import matplotlib.colors as mcolors
import scipy.sparse as sp
from liana.mt import rank_aggregate
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean

In [1]:
counts_data = pd.read_csv("counts_data.csv", index_col=0)
counts_data.columns = counts_data.columns.str.replace('^SeuratProject', '', regex=True)
counts_data = counts_data.T
metadata = pd.read_csv("metadata.csv", index_col=0)
metadata.index = metadata.index.str.replace('^SeuratProject', '', regex=True)

adata = ad.AnnData(X=counts_data.values, obs=metadata, var=pd.DataFrame(index=counts_data.columns))
adata = adata[adata.obs['sample'] == 222107]

types = pd.read_csv('adpkd_clustering.csv')
types['Barcode'] = types['Barcode'] + '_2'
types.index = types['Barcode']
types = types.drop('Barcode', axis = 1)
adata.obs['cell_types'] = types['Kidney group 222107 analysis']

adata = adata[adata.obs['cell_types'].isna() == False]

In [ ]:
adata.X = adata.X.astype(float)
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)

In [ ]:
adata.raw = adata

cellphonedb(adata,
            groupby='cell_types',
            resource_name='cellphonedb',
            expr_prop=0.1,
            verbose=True, key_added='cpdb_res')

In [ ]:
li.mt.rank_aggregate(adata,
                     groupby='cell_types',
                     resource_name='cellphonedb',
                     expr_prop=0.05,
                     verbose=True)

In [ ]:
coi_cellphone = adata.uns['cpdb_res'][adata.uns['cpdb_res']['source'] == 'COI']

In [ ]:
coi_cellphone = coi_cellphone[(coi_cellphone['cellphone_pvals'] <= 0.05) & (coi_cellphone['cellphone_pvals'] > 0)]

In [ ]:
coi_cellphone[['source', 'target', 'ligand_complex','receptor_complex','cellphone_pvals','lr_means']]

In [ ]:
interactions = {
    ('THBS1', 'ITGAV_ITGB3'),
    ('THBS1', 'ITGA3_ITGB1'),
    ('MMP2' ,'ITGAV_ITGB3'),
    ('LAMC1' , 'ITGA7_ITGB1'),
    ('LAMC1' , 'ITGA6_ITGB1'),
    ('LAMC1' , 'ITGA2_ITGB1'),
    ('EDN2','EDNRA'),
    ('FAM3C', 'LAMP1'),
    ('FN1', 'ITGAV_ITGB1'),
    ('FN1', 'ITGAV_ITGB3'),
    ('HBEGF', 'ERBB4'),
    ('JAM3' , 'ITGAM_ITGB2'),
    ('APP', 'CD74'),
    ('COL1A1', 'ITGA2_ITGB1')}

def filter_interaction(row):
    return (
        (row['cellphone_pvals'] <= 0.05) and
        (row['cellphone_pvals'] > 0.0) and
        ((row['ligand_complex'], row['receptor_complex']) in interactions))

In [ ]:
li.pl.dotplot(
    adata=adata,
    colour='lr_means',
    size='cellphone_pvals',
    inverse_size=True,
    source_labels=['COI'],
    target_labels=['DCT', 'Res Mac', 'B cell', 'Mac', 'DC', 'ENDO', 'Mes', 'SM/Fib', 'PODO', 'IC-B'],
    figure_size=(7, 6),
    filter_fun=filter_interaction,
    uns_key='cpdb_res')